In [29]:
import pandas as pd
df = pd.read_csv("../input/MSampleSubmissionStage1.csv").head(2278)
df = df.reset_index()

In [30]:
def which(x):
    return x.split("_")[1]
df.ID = df.ID.apply(which)
df.groupby("ID")["Pred"].count()
#every possible matchup

ID
1107    67
1112    66
1116    65
1124    64
1125    63
        ..
1452     5
1455     4
1458     3
1459     2
1461     1
Name: Pred, Length: 67, dtype: int64

In [40]:
import os
os.chdir("../input")

In [85]:
tf = pd.read_csv("MTeams.csv")
cities_df = pd.read_csv("Cities.csv")


,CityID,City,State
0,4001,Abilene,TX
1,4002,Akron,OH
2,4003,Albany,NY
3,4004,Albuquerque,NM
4,4005,Allentown,PA
...,...,...,...
448,4471,Midland,TX
449,4472,Riviera Maya,MX
450,4473,St. James,JA
451,4474,Victoria,BC


In [ ]:
#rf.WLoc = rf["WLoc"].replace("N",0).replace("H",1).replace("A",0)
#rf["score"] = .5 + (rf["WScore"]-rf["LScore"])*2/(rf["WScore"]) - rf.WLoc/20
#rf.score = rf.score.apply(lambda x: min(x,1)).apply(lambda x: max(x,.5))
#X = X.select_dtypes("int")

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except ValueError:
        return int(x[:-1])

In [364]:
train = pd.read_csv("MNCAATourneyCompactResults.csv")
seed_df = pd.read_csv("MNCAATourneySeeds.csv")
train = train.rename(columns={"WTeamID":"ATeamID","WScore":"AScore","LTeamID":"BTeamID","LScore":"BScore"})
train["y"] = 1
switch_train = train.rename(columns={"BLoc":"ALoc","ATeamID":"BTeamID","AScore":"BScore","BTeamID":"ATeamID","BScore":"AScore"})
switch_train.y = 0
train = pd.concat([train,switch_train])
train = train.merge(seed_df, left_on=["ATeamID","Season"], right_on=["TeamID","Season"])
train = train.merge(seed_df, left_on=["BTeamID","Season"], right_on=["TeamID","Season"])
train = train.rename(columns={"Seed_x":"ASeed","Seed_y":"BSeed"}).drop(columns=["TeamID_x","TeamID_y"])
train.loc[:,["AZone","ASeed"]] = list(train.ASeed.apply(lambda x: [x[0],convert_int(x[1:])]))
train.loc[:,["BZone","BSeed"]] = list(train.BSeed.apply(lambda x: [x[0],convert_int(x[1:])]))


In [365]:
y_train = train["y"]
train = train[["Season","ATeamID","BTeamID","ASeed","BSeed"]] #,"AZone","BZone"


In [366]:
#X[["ATeamID","BTeamID"]] = X[["ATeamID","BTeamID"]].astype("object")
#X[["ATeamID","BTeamID"]] = X[["ATeamID","BTeamID"]].astype("object")
test = pd.read_csv("../input/MSampleSubmissionStage1.csv")
test_ID = test.ID
test = pd.DataFrame(list(test.ID.apply(lambda x: x.split("_"))), test_ID, columns=["Season","ATeamID","BTeamID"]).astype("int").reset_index()
test = test.merge(seed_df, left_on=["ATeamID","Season"], right_on=["TeamID","Season"])
test = test.merge(seed_df, left_on=["BTeamID","Season"], right_on=["TeamID","Season"])
test = test.rename(columns={"Seed_x":"ASeed","Seed_y":"BSeed"}).drop(columns=["TeamID_x","TeamID_y"])
test.loc[:,["AZone","ASeed"]] = list(test.ASeed.apply(lambda x: [x[0],convert_int(x[1:])]))
test.loc[:,["BZone","BSeed"]] = list(test.BSeed.apply(lambda x: [x[0],convert_int(x[1:])]))
test = test.set_index("ID").loc[test_ID]

test = test[["Season","ATeamID","BTeamID","ASeed","BSeed"]]




In [367]:
X = pd.concat([train,test])
X = pd.get_dummies(X)
train = X.iloc[:len(y_train)]
test = X.iloc[len(y_train):]

In [368]:
from sklearn.utils import shuffle
train, y_train = shuffle(train, y_train)
train

,Season,ATeamID,BTeamID,ASeed,BSeed
690,1990,1261,1437,5,12
582,1989,1438,1344,5,12
140,1986,1298,1181,7,1
2362,2003,1122,1257,13,4
2318,2003,1338,1266,2,3
...,...,...,...,...,...
2695,2006,1211,1231,3,6
3101,2009,1113,1393,6,3
2772,2006,1326,1172,2,15
2330,2003,1231,1338,7,2


In [330]:
#X[X.isnull().any(axis=1)]

In [369]:
import numpy as np
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler # Preprocessing
seed = 5
n_folds =5
scoring='neg_mean_squared_error'
model = LGBMRegressor()
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
np.sqrt(-cross_val_score(model, train, y_train, cv= kfold,
                                 scoring=scoring, n_jobs=1))

array([0.45526816, 0.46716656, 0.43417769, 0.46396829, 0.46214982,
       0.44983673, 0.41469122, 0.44489021, 0.45887669, 0.43514039])

In [359]:
model.fit(train, y_train)

LGBMRegressor()

In [360]:
finalPred = model.predict(test)

In [361]:
sub = pd.DataFrame()
sub['ID'] = test_ID
sub['Pred'] = finalPred
sub.to_csv('../output/lgb_submission.csv',index=False)

In [370]:
test

,Season,ATeamID,BTeamID,ASeed,BSeed
2015_1107_1112,2015,1107,1112,14,2
2015_1107_1116,2015,1107,1116,14,5
2015_1107_1124,2015,1107,1124,14,3
2015_1107_1125,2015,1107,1125,14,15
2015_1107_1129,2015,1107,1129,14,11
...,...,...,...,...,...
2019_1449_1459,2019,1449,1459,9,7
2019_1449_1463,2019,1449,1463,9,14
2019_1458_1459,2019,1458,1459,5,7
2019_1458_1463,2019,1458,1463,5,14
